# Testing the `MultivariateCoupledNormal` class and entropy functions with $\kappa = 0$

Use `!pip install -i https://test.pypi.org/simple/ nsc-test` to install the latest version of `nsc-test`.

In [1]:
!pip install -i https://test.pypi.org/simple/ nsc-test

Looking in indexes: https://test.pypi.org/simple/


# Setting up

In [2]:
import nsc_test
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.stats import multivariate_normal
from scipy.stats import t
from nsc_test.distributions import MultivariateCoupledNormal
from nsc_test.math.entropy import importance_sampling_integrator
from nsc_test.math.entropy_norm import coupled_normal_entropy, biased_coupled_probability_norm, coupled_probability_norm, coupled_cross_entropy_norm, coupled_entropy_norm, coupled_kl_divergence_norm
import matplotlib.pyplot as plt

import site
from importlib import reload
reload(site)
import sys #Changing the system path
sys.path.insert(0, '/home/hongxiang/Documents/repos/Nonlinear-Statistical-Coupling')

from nsc.distributions.multivariate_coupled_normal_tf import MultivariateCoupledNormal as mvn_tf

Importing NSC lib v0.0.0.13.


2021-08-08 15:09:37.009763: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-08 15:09:37.009815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


KeyboardInterrupt: 

In [ ]:
# Set the locs.
loc = np.array([[0., 0.,],
                [100., -100.],
                [-10., 5.]])
# Set the scales.
scale = np.array([[1., 1.],
                  [10., 10.],
                  [1., 1.]])

# Find the number of batches.
n_batches = loc.shape[0]

# Set the kappa.
kappa = 0.0

# Initialize a MultivariateCoupledNormal object.
mvn = MultivariateCoupledNormal(loc=loc, scale=scale, kappa=kappa)

In [ ]:
loc_tensor = tf.convert_to_tensor(loc, dtype=tf.float32)
scale_tensor = tf.convert_to_tensor(scale, dtype=tf.float32)
cov_tensor = tf.convert_to_tensor(mvn._scale, dtype=tf.float32)
mvn_tf_dist = mvn_tf(loc=loc_tensor, scale=scale_tensor, kappa=0.01)

In [ ]:
#mvn_tf = tfp.distributions.MultivariateNormalTriL(loc=loc_tensor, scale_tril=cov_tensor)

# Testing `sample_n`

In [ ]:
# Set the number of samples per batched distribution.
n = 10000
# Sample n observations from each batched distribution.
samples = mvn.sample_n(n)
samples_tf = mvn_tf_dist.sample_n(n)

print(f"Expected dimensions of samples: {loc.shape[0], n, loc.shape[1]}")
print(f"Actual dimensions of samples: {samples.shape}")
print(f"Actual dimensions of samples_tf: {samples_tf.shape}")

In [ ]:
for i in range(n_batches):
    print(f"Mean of distribution 1: {tuple(loc[i])}")
    print(f"Mean of samples from distribution 1: {tuple(samples[i].mean(axis=0))}")
    print(f"Std. Dev. of distribution 1: {tuple(scale[i])}")
    print(f"Std. Dev. of samples from distribution 1: {tuple(samples[i].std(axis=0))}")
    print("\n")

In [ ]:
for i, batch in enumerate(samples_tf):
    mean_temp = tuple(tf.math.reduce_mean(batch, axis=0).numpy())
    std_temp = tuple(tf.math.reduce_std(batch, axis=0).numpy())
    
    print(f"Mean of distribution 1: {tuple(loc[i])}")
    print(f"Mean of samples from distribution 1: {mean_temp}")
    print(f"Std. Dev. of distribution 1: {tuple(scale[i])}")
    print(f"Std. Dev. of samples from distribution 1: {std_temp}")
    print("\n")

In [ ]:
for i in range(samples.shape[0]):
    x, y = samples[i, :,0], samples[i, :,1]
    plt.scatter(x, y,
                color="blue", alpha=0.1)
    plt.show()

# Testing `prob` with batches (multiple distributions)

In [ ]:
mvn_probs = mvn.prob(samples)

# Loop through the 
for i in range(n_batches):
    # Get the i-th distributions sample densities and reshape them to be 1-D.
    dist_1_probs = mvn_probs[i].reshape(-1,)
    # Initialize a scipy multivariate_normal object.
    mvn_scipy_1 = multivariate_normal(loc[i], scale[i]**2)
    # Get the probability densities from scipy object.
    scipy_probs_1 = mvn_scipy_1.pdf(samples[i])
    
    print(f"The densities from the Coupled MVN and SciPy MVN are close: {np.allclose(dist_1_probs, scipy_probs_1)}")
    print("\n")

# `importance_sampling_integrator`

In [ ]:
importance_sampling_integrator(mvn.prob, mvn.prob, mvn.sample_n, n=10000, seed=1)

# `coupled_normal_entropy`

In [ ]:
#coupled_normal_entropy(mvn._sigma, kappa=0.0)

In [ ]:
#coupled_normal_entropy(mvn._sigma, kappa=0.003)

In [ ]:
#mvn._sigma

In [ ]:
#0.5 * np.log((2*np.pi*np.exp(1)) ** mvn.loc[0].shape[0] * np.linalg.det(mvn._sigma[0]))

# `biased_coupled_probability_norm`

In [ ]:
new_mvn = biased_coupled_probability_norm(mvn, kappa=0.1, alpha=2)

print(new_mvn.loc)
print(new_mvn._sigma)
print(new_mvn.kappa)

In [ ]:
new_mvn_samples = new_mvn.sample_n(100)

# `coupled_probability_norm`

In [ ]:
new_mvn_pdf = coupled_probability_norm(mvn, kappa=0.1, alpha=2.0)

np.all(new_mvn_pdf(new_mvn_samples) == new_mvn.prob(new_mvn_samples))

# `coupled_cross_entropy_norm`

In [ ]:
coupled_cross_entropy_norm(
    mvn,
    new_mvn,
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

# `coupled_entropy_norm`

In [ ]:
coupled_entropy_norm(
    mvn,
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

In [ ]:
true_entropies = [
    0.5 * np.log((2*np.pi*np.exp(1)) ** mvn.loc[i].shape[0] * np.linalg.det(mvn._sigma[i])) for i in range(mvn._batch_shape[0])
]
true_entropies = np.array(true_entropies)
true_entropies.reshape(mvn._batch_shape[0], 1, 1)

# `coupled_kl_divergence_norm`

In [ ]:
coupled_kl_divergence_norm(
    mvn, 
    new_mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

In [ ]:
coupled_kl_divergence_norm(
    mvn, 
    mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)

In [ ]:
coupled_kl_divergence_norm(
    new_mvn, 
    new_mvn, 
    kappa=0.0, 
    alpha=2.0, 
    root=False,
    n=10000,
    seed=1
)